In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN2002"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
import sys
from hdf5storage import savemat
import pickle

sys.path.append('../../../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval as nn_eval 

## Loading synthetic data

In [4]:
#loading data
data_path= '/data2/data/zebrahim/synthetic_dataset/data_8192_1000_3_4_4_.03_.03_.2' + 'processed_data.p'

with open(data_path, 'r') as fin:
    data = pickle.load(fin)
    
train_data = data['train_data']
validation_data = data['validation_data']
test_data = data['test_data']

train_gt = np.expand_dims(data['train_gt'], axis=-1)
validation_gt = np.expand_dims(data['validation_gt'], axis=-1)
test_gt = np.expand_dims(data['test_gt'], axis=-1)

index_of_changes = data['index_of_changes']
 

In [5]:
#index of changes for test dataset
gt_test_idx_changes = index_of_changes[900:1000]

## Model

In [6]:
# Model parameters
n_variables = 12
learning_rate = 0.001
batch_size = 10


wavelet_levels = 6    #number of wavelet layer to have in the network
convolution_levels = 6    #number of convolution to have in the network
pooling_stride = [4, 2, 2, 2, 1, 1]    #stride of pooling in convolution
weights_shape = [[7, n_variables, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32],
                 [7, 32, 32]]
bias_shape = [32, 32, 32, 32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
granularity = np.prod(pooling_stride)
n_classes = 1

##positive weight for weighted cross entropy
s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()

In [7]:
print pos_weight

3.18601822124


In [8]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [9]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/' + exp_name 

In [10]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [11]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation)
                        

In [12]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = validation_data,
                    ground_truth_validation = validation_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size)

In [13]:
trainer.train(max_iter = 10000,
                train_eval_step = 100,
                validation_eval_step = 100,
                display_step = 100)

Tarin Iter 0, auc= 0.410122, fmeasure_max= 0.422018, precision= 0.355017, recall= 0.520194
Train Random guess:  , auc= 0.239512fmeasure_max= 0.389921, precision= 0.242175, recall= 1.000000
Validation Iter 0, auc= 0.329125, fmeasure_max= 0.432797, precision= 0.310268, recall= 0.715266
Validation Random guess:  auc= 0.234186, fmeasure_max= 0.371740, precision= 0.230172, recall= 0.965695
Minibatch Loss= 1.071120
Tarin Iter 100, auc= 0.764381, fmeasure_max= 0.712610, precision= 0.691323, recall= 0.735250
Train Random guess:  , auc= 0.263452fmeasure_max= 0.411453, precision= 0.259013, recall= 1.000000
Validation Iter 100, auc= 0.718941, fmeasure_max= 0.695510, precision= 0.663551, recall= 0.730703
Validation Random guess:  auc= 0.226166, fmeasure_max= 0.371574, precision= 0.228180, recall= 1.000000
Minibatch Loss= 0.796884
Tarin Iter 200, auc= 0.832989, fmeasure_max= 0.782365, precision= 0.735526, recall= 0.835575
Train Random guess:  , auc= 0.265388fmeasure_max= 0.416174, precision= 0.2627

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [14]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

Testing auc = 0.922551 F1_max = 0.838865 prec = 0.824364 rec = 0.853885
Test Random guess: auc= 0.230127 fmeasure_max= 0.375742 , precision= 0.231331 , recall= 1.000000


In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})